## Load Conversations

In [1]:
import sys
sys.path.append('.')

import pandas as pd
from utils import load_jsonl, dump_jsonl
# import numpy as np

In [2]:
# "AA"

In [3]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
    
    
    def to_message_str(messages, users):
        s = []
        u = []
        for m in messages:
            if users[m['user_id']] == "SYS":
                s.append(m['text'])
            else:
                u.append(m['text'])
                
        return s, u
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        chunk_size = 100
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            s, u = to_message_str(sub_messages, users)
            
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            label = row[col_label]
                
            newdata.append({
                "user": u,
                "sys": s,
                "label": label,
                "nturn": len(sub_messages)
            })
        
#     n_val = int(len(newdata)*0.05)
#     n_test = n_val
    
#     test = newdata[0:n_test]
#     val = newdata[n_test:n_test+n_val]
#     train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [4]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
df.head()

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",1. Close,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",1. Close,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [5]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,551,551,551
2. Know each other,230,230,230
3. Don't know each other,435,435,435
4. Don't like each other,5,5,5


In [6]:
# df.groupby("label").count()

In [7]:
def get_task2_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}")
        
    newdata = []
    for row in conversations:
        row["messages"].sort(key=lambda x: x["created_at"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())!=0 else "SYS"
                users[m["user_id"]] = username
                
        if len(users)>2:
            print("More than 1 users", len(users))
        
        
        
        messages = row["messages"]
        s = []
        u = []
        for m in messages:
            text = m['text'].replace("[USR]", "").replace("[URL]", "URL")
            if users[m['user_id']] == "SYS":
                s.append(text)
            else:
                u.append(text)
        
        label = row[col_label]
            
        if pd.isna(label):
            continue
        
        if label in skips:
            continue
                
        newdata.append({
            "user": u,
            "sys": s,
            "label": label,
            "nturn": len(messages)
        })
                
            
    n_val = int(len(newdata)*0.1)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata)

In [8]:
df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority")
df.head()

Loaded 2486 records from ./Task2/annotated/annotated.jsonl


,user,sys,label,nturn
0,[ แตงโม ชวนแฟนไปค่ะ ไม่ใช่กระติกชวนไป กลับไปหา...,[ทบทวน\nงานคืนนั้น #กระติก ชวนแฟนพี่ตม.ไปด้วยน...,3. Not respect,3
1,[ รูปนี้จ้า ยัยน้องตัวเล็กของพี่มิว☀️🌻\n \n#...,[เนื่ิองจากวันนี้...เป็นวันครบรอบ 3 ปี\nมาเล่น...,1. Respect,3
2,[ แอบคิดเหมือนกัน้ลยค่ะว่าจะมีอีกรอบ],[เธรดนี้จะมาวิเคราะห์น้ำท่วมปี54 ที่จริงมันอาจ...,2. Normal,3
3,[ เนี่ยสิ่งที่กูพูด URL],[มาดิ้นควยไรชองมึง\nมึงอ่ะโชว์โง่ไอ้สัตว์ เสร่...,3. Not respect,3
4,"[ สรุปปวินนี่บล็อกคนด้วยมั้ย?, สรุป ที่น่าโมโ...",[แต่ปวินกับไพรวัลย์ก็ไม่เข้าใจนะเหมือนเจ้เกลีย...,2. Normal,5


In [9]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Respect,319,319,319
2. Normal,1661,1661,1661
3. Not respect,364,364,364


In [10]:
df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness")
df.head()

Loaded 1221 records from ./Task3/annotated/annotated.jsonl


,user,sys,label,nturn
0,"[ดีๆวาย, เป็นยังไงบ้างช่วงนี้, รวยๆ เฮงๆ , เหม...","[ว่าไงปราง, เหมือนเดิม ขายของ นอนตื่น ขายของ ช...",1. Close,35
1,"[วันนี้เรามาพูดถึงเรื่องภาพยนตร์กัน, ปกฅิแล้วค...","[โอเคค่ะ, ที่บ้านค่ะ, ดูแทบทุกวันเลย, ดูได้ทุก...",2. Know each other,39
2,"[เธอ, สรุป อ้วนจะซื้อรถเมื่อไหร่, แล้วอ้วนดูรถ...","[อะไรหมู, กลางปี, ว่าจะเอา ativ ตัวใหม่, ว่าจ...",2. Know each other,40
3,"[มึง, แมวมึงเป็นไงกันบ้างอ่ะ, ตอนนี้มีกี่ตัวนะ...","[ว่า, เด็กๆหรอ, ซนปกติเลยมึง, 5ตัว, แค่บ้านกูน...",1. Close,35
4,"[อ้วน, อยากได้แมว, อยากเลี้ยง, เอาไว้ตอนตัวไม่...","[ไรหมู, เอาไปทำไมแมว, จะหาจากไหน, เดี๋ยวถามให้...",1. Close,39


In [11]:
df.groupby("label").count()

,user,sys,nturn
label,,,
1. Close,462,462,462
2. Know each other,696,696,696
3. Don't know each other,52,52,52
4. Don't like each other,11,11,11


## Load Lexicons

In [12]:
from pythainlp.tokenize import word_tokenize

In [13]:
import json
with open("./PrivateSpace/thai-dictionary/RoyalInstituteDictionary/words.json", encoding="utf-8") as fin:
    raw = json.load(fin)
    thaidict_royal = set()
    for k in raw:
        thaidict_royal.update(raw[k])

In [14]:
lexicons_arr = load_jsonl("lexicons.jsonl")

Loaded 25573 records from lexicons.jsonl


In [15]:
from collections import defaultdict
tags = set()
lexicons = {}
lexicons_keys = defaultdict(list)

for key, values  in lexicons_arr:
    if len(key) <= 1:
        continue
        
    key = key.lower()
    if key.endswith("rep"):
        key = key.replace("rep", "")
        
    w = word_tokenize(key)
    
    lexicons_keys[w[0]].append(key)
    
    tag = [t for t in values["tags"] if not t.startswith("cat:")]
    lexicons[key] = tag
    tags.update(tag)

In [16]:
from pythainlp.util import countthai

In [254]:
import re
from itertools import groupby
import emoji
from pythainlp.util import countthai
from tqdm import tqdm

def rm_reptitive(text):
    s = ""
    groups = groupby(text)
    for label, group in groups:
        g = list(group)
        if len(g) >= 3:
            s += f"{label*3} rep "
        else:
            s += "".join(g)
    return s

def remove_space(sent):
    newwords = []
    for w in sent:
        if len(w.strip())==0:
            continue
        newwords.append(w)
    return newwords

import re
def notthai(w):
    if countthai(w) > 50:
        return False
    
    if w in ["usr", "sys", "rep"]:
        return False
    
    nt = re.sub(r'\W+', '', w)
    if len(nt) > 0 and not nt.isnumeric():
#         print(nt)
        return True
    return False

def analyse_conv_per_person(texts):
    
    # Word Statistic
    texts = [t.lower() for t in texts]
    texts = [rm_reptitive(t) for t in texts]
    words = [word_tokenize(t) for t in texts]
    words = [remove_space(w) for w in words]
    
    nlongword = 0
    ndict = 0
    nnotthai = 0
    nthai = 0
    for sent in words:
        ndict += sum([1 if w in thaidict_royal else 0 for w in sent])
        nlongword += sum([1 if len(w) > 7 else 0 for w in sent])
        nthai += sum([1 if countthai(w) > 50 else 0 for w in sent])
            
        nnotthai += sum([1 if notthai(w) else 0 for w in sent])
    
    uwords = set()
    for sent in words:
        uwords.update(sent)
    
    # Lexicon 
    lex = []
    for sidx, sent in enumerate(words):
        for widx, w in enumerate(sent):
            if w not in lexicons_keys:
                continue 
            
            s = "".join(sent[widx:])
            for l in lexicons_keys[w]:
                if not s.startswith(l):
                    continue

                lex.extend(lexicons[l])
#                 print(">>", w, l, lexicons[l])

    lexcat = {}
    for l in lex:
        if l not in lexcat:
            lexcat[l] = 0
        lexcat[l] += 1
    
    # Stylistic words
    nrepeat = 0
    for sidx, sent in enumerate(words):
        nrepeat += sum([1 if w=="rep" else 0 for w in sent])
    
    s = " ".join(texts)
    nemoji = emoji.emoji_count(s)
    

    return {
        "nsent": len(texts),
        "nword": sum([len(w) for w in words]),
        "ndict": ndict,
        "nunique": len(uwords),
        "nlongword": nlongword,
        "nrepeat": nrepeat,
        "nthai": nthai,
        "nnotthai": nnotthai,
        "nemoji": nemoji,
        **lexcat
    }
  


def analyse_conversation(df):
    metrics = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        ms = analyse_conv_per_person(row["sys"])
        mu = analyse_conv_per_person(row["user"])
        metrics.append((ms, mu))
        
    
    return metrics

analyse_conv_per_person(["เมิงงงงงงงมันโง่เหมือนควายยยยยยยยย"]);

## Run Analysis

In [255]:
import pickle
def save_obj_values(filepath, obj):
  with open(filepath, 'wb') as fin:
    pickle.dump(obj, fin)
    
def load_obj_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

In [256]:
analysis_values = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

analysis_labels = {
    "closeness": [None, None, None],
    "authority": [None, None, None],
}

In [258]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "closeness", skips = [])
metrics = analyse_conversation(df)
analysis_values["closeness"][0] = metrics
analysis_labels["closeness"][0] = df["label"].values


df = get_task2_conver("./Task2/annotated/annotated.jsonl", "closeness", skips = [])
metrics = analyse_conversation(df)
analysis_values["closeness"][1] = metrics
analysis_labels["closeness"][1] = df["label"].values


df = get_task1_conver("./Task3/annotated/annotated.jsonl", "closeness", skips = [])
metrics = analyse_conversation(df)
analysis_values["closeness"][2] = metrics
analysis_labels["closeness"][2] = df["label"].values

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:05<00:00, 217.75it/s]


Loaded 2486 records from ./Task2/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2124/2124 [00:04<00:00, 443.72it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:05<00:00, 225.04it/s]


In [261]:
df = get_task1_conver("./Task1/annotated_conersations.jsonl", "authority", skips = [])
metrics = analyse_conversation(df)
analysis_values["authority"][0] = metrics
analysis_labels["authority"][0] = df["label"].values


df = get_task2_conver("./Task2/annotated/annotated.jsonl", "authority", skips = [])
metrics = analyse_conversation(df)
analysis_values["authority"][1] = metrics
analysis_labels["authority"][1] = df["label"].values


df = get_task1_conver("./Task3/annotated/annotated.jsonl", "authority", skips = [])
metrics = analyse_conversation(df)
analysis_values["authority"][2] = metrics
analysis_labels["authority"][2] = df["label"].values

Loaded 1234 records from ./Task1/annotated_conersations.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1226/1226 [00:05<00:00, 225.54it/s]


Loaded 2486 records from ./Task2/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2344/2344 [00:05<00:00, 456.82it/s]


Loaded 1221 records from ./Task3/annotated/annotated.jsonl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1221/1221 [00:05<00:00, 227.18it/s]


In [263]:
# save_obj_values("analysis_values.pkl", analysis_values)
# save_obj_values("analysis_labels.pkl", analysis_labels)

In [264]:
analysis_values = load_obj_values("analysis_values.pkl")
analysis_labels = load_obj_values("analysis_labels.pkl")

## Appendix C: Descriptive Analytics

In [265]:
metric_names = {
    "Conversational Statistics" : {
        "nsent": "Number of utterance",
        "nword": "Number of word",
    },
    "Linguistic Complexity" : {
        "nunique": "Vocabulary size",
        "nthai": "Thai words",
        "nnotthai": "Non-Thai words",
        "nlongword": "Long words",
        "ndict": "Dictionary words",
        "transliterated": "Transliteration",
    },
    "Pronoun": {
        "pronoun": "All pronoun",
        "pronoun_1st": ">> 1st person pronoun",
        "pronoun_2nd": ">> 2nd person pronoun",
        "pronoun_3rd": ">> 3rd person pronoun",
        "pronoun_misspelling": ">> Pronoun in non-standard spelling",
    },
    
    "Sentence-ending Particles": {
        "particles": "All particles",
        "particles_SARP": ">> Socially-related particles",
        "particles_notSARP": ">> Non-socially-related particles",
        "particles_misspelling": ">> Particle in non-standard spelling",
    },
    
    "Sentiment-related": {
        "sentiment": "Sentiment words",
        "sentiment_positive": ">> Positive words",
        "sentiment_negative": ">> Negative words",
    },
    
    "Spelling Variation": {
        "misspelling": "All spelling variation",
        "misspelling_common": ">> Common misspelt words",
        "misspelling_intention": ">> Morphophonemic variation",
        "misspelling_shorten": ">> Simplified variation",
        "nrepeat": ">> Repeated characters",
        "nemoji": ">> Emoji",
#         "abbr": "Abbreviation",
#         "slang": "Slang",
#         "swear": "Swear words"
    }
}

In [266]:
clse_print_labels = ['1. Close', '2. Know each other', "3. Don't know each other", "4. Don't like each other"]
auth_print_labels = ['0. Very respect', '1. Respect', '2. Normal', '3. Not respect']

In [269]:
def print_lexi_stat(analysis_values, analysis_labels, factor, setting, print_labels):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    printed_text = ""
    rows = []
    for (ms, mu), l in zip(values, labels):
        for m in mu:
            if m in ["nsent", "nword"]:
                v = mu[m]
            else:
#                 v = mu[m]*100/mu["nword"]
                v = mu[m]
                
            rows.append({
                "label": l,
                "metric": m,
                "value": v
            })
        
        if "particles_SARP" in mu:
            particles_notSARP = mu["particles"] - mu["particles_SARP"]
            rows.append({
                "label": l,
                "metric": "particles_notSARP",
                "value": particles_notSARP
            })

    df = pd.DataFrame(rows).groupby(["label", "metric"]).mean().reset_index()
    
    for g in metric_names:
        # print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{5}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for l in print_labels:
                row = df[(df["label"]==l) & (df["metric"]==m)]
#                 print(l, m)
#                 print(df)
                if len(row)!=0:
                    s += f"& {row['value'].values[0]:.2f} "
                else:
                    s += f"& - "

            s += "\\\\"
            # print(s)
            printed_text += "    "+s+"\n"
        
        printed_text += "    &  & &  & \\\\"+"\n"
        printed_text += "\hline"+"\n"
        
        # print("&  & &  & \\\\")
        # print("\hline")
    # print(printed_text)
    return df, printed_text

In [270]:
df, printed_text1 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 0, clse_print_labels)

In [271]:
df, printed_text2 = print_lexi_stat(analysis_values, analysis_labels, "authority", 0, auth_print_labels)

In [272]:
df, printed_text3 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 1, clse_print_labels)

In [273]:
df, printed_text4 = print_lexi_stat(analysis_values, analysis_labels, "authority", 1, auth_print_labels)

In [274]:
df, printed_text5 = print_lexi_stat(analysis_values, analysis_labels, "closeness", 2, clse_print_labels)

In [275]:
# set(analysis_labels["authority"][2])

In [276]:
df, printed_text6 = print_lexi_stat(analysis_values, analysis_labels, "authority", 2, auth_print_labels)

In [279]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (printed_text1, printed_text2),
    (printed_text3, printed_text4),
    (printed_text5, printed_text6),
]

printed_text = ""
for section, (t1, t2) in zip(sections, table_contents):
    
    printed_text += "\subsection{"+section+"}"+"\n\n"
    
    printed_text += '''\subsubsection{Closeness}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline

    Linguistic Features & Close & Know each other & Don't know each other &  Don't like each other\\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t1).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"

    printed_text += '''\clearpage
\subsubsection{Respect}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.16\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline

    Linguistic Features & Very respect & Respect & Normal &  Not respect\\\\
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t2).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"
    printed_text += "\clearpage"
    printed_text += "\n\n"
    # break
    
# print(printed_text)

In [287]:
# print(printed_text)

## Regression w/ Othogonal Polynomial Coding
https://medium.com/@wyess/demystifying-statistical-analysis-3-the-one-way-anova-expressed-in-linear-regression-99269e84edd9

In [288]:
# def closeness_to_vec1(label):
#     if label == "1. Close":
#         return {"x1": 1, "x2": 0}
#     elif label == "2. Know each other": ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == "3. Don't know each other": 
#         return {"x1": 0, "x2": 1}
#     elif label == "4. Don't like each other":
#         return None
#     else:
#         return None
    
# def authority_to_vec1(label):
#     if label == '0. Very respect':
#         return {"x1": 1, "x2": 0}
#     elif label == '1. Respect': ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == '2. Normal':
#         return {"x1": 0, "x2": 1}
#     elif label == '3. Not respect':
#         return None
#     else:
#         return None

def closeness_to_vec1(label):
    if label == "1. Close":
        return {"x1": 1, "x2": -1}
    elif label == "2. Know each other":
        return {"x1": 0, "x2": 2}
    elif label == "3. Don't know each other": 
        return {"x1": -1, "x2": -1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec1(label):
    if label == '0. Very respect':
        return {"x1": 1, "x2": -1}
    elif label == '1. Respect': 
        return {"x1": 0, "x2": 2}
    elif label == '2. Normal':
        return {"x1": -1, "x2": -1}
    elif label == '3. Not respect':
        return None
    else:
        return None

In [289]:
clse_coef_labels = ["b", "a1", "a2"]
auth_coef_labels = ["b", 'a1', 'a2']

In [290]:
def to_features(analysis_values, analysis_labels, factor, setting, to_vec_func, skips=[]):
    values = analysis_values[factor][setting]
    labels = analysis_labels[factor][setting]
    assert(len(values)==len(labels))
    
    rows = []
    for (ms, mu), l in zip(values, labels):
        if l in skips:
            continue
            
        x = to_vec_func(l)
        if x is None:
            continue
        
        for m in mu:
            if m in ["nsent", "nword"]:
                v = mu[m]
            else:
                v = mu[m]*100/mu["nword"]
            
            
            rows.append({
                "metric": m,
                "value": v,
                **x
            })
            
        if "particles_SARP" in mu:
            particles_notSARP = mu["particles"] - mu["particles_SARP"]
            rows.append({
                "metric": "particles_notSARP",
                "value": particles_notSARP,
                **x
            })
            
    feats = pd.DataFrame(rows)  
    return feats

In [291]:
import statsmodels.api as sm
import numpy as np

def print_anova_test(feats, n=3):
    feat_names = feats["metric"].unique()
    
    feat_important = []
    for f in feat_names:
        if "cat:" in f:
            continue

        try:
            d = feats[feats["metric"]==f]
        #     print(f, len(d))
            x_columns = [f"x{i+1}" for i in range(n)]
            X = d[x_columns]
            X = sm.add_constant(X)
            Y = d["value"]
            model = sm.OLS(Y,X)
            results = model.fit()
            
            ncoef = n + 1
            
            t_test = results.t_test(np.identity(ncoef))
            f_test = results.f_test(np.identity(ncoef))
            o = {"feat": f, "f_value": f_test.pvalue}
            for i in range(ncoef):
                o[f"coef{i}"] = t_test.effect[i]
                
            for i in range(ncoef):
                o[f"pval{i}"] = t_test.pvalue[i]
                
            feat_important.append(o)
        except Exception as e:
            print("error", f, e)

    outputs = pd.DataFrame(feat_important)
    return outputs

In [292]:
# def print_weights(outputs, coef_labels, labels, base_category=None):
    
#     coefs = {}
#     non_coefs = {}
#     for i, label in enumerate(coef_labels):
#         if i==0:
#             continue
            
#         cond = outputs[f"pval{i}"] < 0.05
#         for _, row in outputs[cond].sort_values(f"coef{i}", ascending=False).iterrows():
#             coefs[(label, row["feat"])] = row[f"coef{i}"]
        
#         cond = outputs[f"pval{i}"] >= 0.05
#         for _, row in outputs[cond].iterrows():
#             non_coefs[(label, row["feat"])] = row[f"coef{i}"]

# #     print(coefs)
#     for g in metric_names:
#         print("\multicolumn{5}{l}{\\textit{"+g+"}} \\\\")
#         for m in metric_names[g]:
#             s = f"{metric_names[g][m]} "
#             for l in labels:
#                 if l == base_category:
#                     s += f"& * "
#                 elif (l, m) in coefs:
#                     val = coefs[(l, m)]
#                     s += "& \cellcolor{gray!25} "+f"{val:.2f} "
#                 elif (l, m) in non_coefs:
#                     val = non_coefs[(l, m)]
#                     s += f"& {val:.2f} "
#                 else:
#                     s += f"& - "

#             s += "\\\\"
#             print(s)
#         print("&  & &  & \\\\")
#         print("\hline")

# # gray!25 < gray!50 < gray!80

In [293]:
def print_weights(outputs):
    
    coefs = {}
    coef_labels = ["b", "a1", "a2"]
    for i, label in enumerate(coef_labels):
        for _, row in outputs.iterrows():
            coefs[(label, row["feat"])] = (row[f"coef{i}"], row[f"pval{i}"])

#     print(coefs)

    printed_text = ""
    for g in metric_names:
        # print("\multicolumn{4}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for l in coef_labels:                    
                val, pval = coefs[(l, m)]
                if l=="b":
                    s += f"& {val:.2f} "
                elif pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                else:
                    s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
        printed_text += "&  & &\\\\" + "\n"
        printed_text += "\hline" + "\n"
        # print("&  & &\\\\")
        # print("\hline")
    return printed_text

In [294]:
feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec1, skips=[])
clse1_poly_outputs = print_anova_test(feats, n=2)
# print_weights(outputs, clse_coef_labels, clse_print_labels, "2. Know each other")
printed_text1 = print_weights(clse1_poly_outputs)

In [295]:
feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec1, skips=["3. Not respect"])
auth1_poly_outputs = print_anova_test(feats, n=2)
# print_weights(outputs, auth_coef_labels, auth_print_labels, "1. Respect")
printed_text2 = print_weights(auth1_poly_outputs)

In [296]:
# def closeness_to_vec2(label):
#     if label == "1. Close":
#         return {"x1": 1, "x2": 0}
#     elif label == "2. Know each other": 
#         return {"x1": 1, "x2": 0}
#     elif label == "3. Don't know each other": ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == "4. Don't like each other":
#         return {"x1": 0, "x2": 1}
#     else:
#         return None
    
# def authority_to_vec2(label):
#     if label == '0. Very respect':
#         return None
#     elif label == '1. Respect': 
#         return {"x1": 1, "x2": 0}
#     elif label == '2. Normal': ## Base category
#         return {"x1": 0, "x2": 0}
#     elif label == '3. Not respect':
#         return {"x1": 0, "x2": 1}
#     else:
#         return None

def closeness_to_vec2(label):
    if label == "1. Close":
        return {"x1": 1, "x2": -1}
    elif label == "2. Know each other": 
        return {"x1": 0, "x2": 2}
    elif label == "3. Don't know each other": ## Base category
        return {"x1": -1, "x2": 1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec2(label):
    if label == '0. Very respect':
        return None
    elif label == '1. Respect': 
        return {"x1": 1, "x2": -1}
    elif label == '2. Normal': ## Base category
        return {"x1": 0, "x2": 2}
    elif label == '3. Not respect':
        return {"x1": -1, "x2": -1}
    else:
        return None

In [297]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec2)
clse2_poly_outputs = print_anova_test(feats, n=2)
# clse_coef_labels = ["b", "2. Know each other", "4. Don't like each other"]
# print_weights(outputs, clse_coef_labels, clse_print_labels, "3. Don't know each other")
printed_text3 = print_weights(clse2_poly_outputs)

error particles_?? wrong shape for coefs


In [298]:
feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec2)
auth2_poly_outputs = print_anova_test(feats, n=2)
# auth2_coef_labels = ["b", '1. Respect', '3. Not respect']
# print_weights(outputs, auth2_coef_labels, auth_print_labels, "2. Normal")
printed_text4 = print_weights(auth2_poly_outputs)

error particles_?? r_matrix performs f_test for using dimensions that are asymptotically non-normal


/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [299]:
# auth2_effect_outputs

In [300]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec2, skips=[])
clse3_poly_outputs = print_anova_test(feats, n=2)
# print_weights(outputs, clse_coef_labels, clse_print_labels, "2. Know each other")
printed_text5 = print_weights(clse3_poly_outputs)

feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec2, skips=[])
auth3_poly_outputs = print_anova_test(feats, n=2)
# print_weights(outputs, auth_coef_labels, auth_print_labels, "1. Respect")
printed_text6 = print_weights(auth3_poly_outputs)

/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [301]:
# sections = [
#     "Setting 1: Private Conversations with Self-Reported Labels",
#     "Setting 2: Public Conversations with Labels from 3rd Party ",
#     "Setting 3: Private Conversations with Labels from 3rd Party ",
# ]

# table_contents = [
#     (printed_text1, printed_text2),
#     (printed_text3, printed_text4),
#     (printed_text5, printed_text6),
# ]

# printed_text = ""
# for section, (t1, t2) in zip(sections, table_contents):
    
#     printed_text += "\subsection{"+section+"}"+"\n\n"
    
#     printed_text += '''\subsubsection{Closeness}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c
# }
#     \hline

#     Lexical Features & b & a1 & a2 \\\\
#     \hline
#     \endfirsthead

#     \endhead
#     '''
    
#     printed_text += ("\n"+t1).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"

#     printed_text += '''\clearpage
# \subsubsection{Respect}
# \\begin{longtable}[h]{
#     p{\dimexpr 0.40\linewidth-2\\tabcolsep}|c|c|c
# }
#     \hline

#     Lexical Features & b & a1 & a2 \\\\
#     \hline
#     \endfirsthead

#     \endhead
#     '''
    
#     printed_text += ("\n"+t2).replace("\n", "\n    ")
#     printed_text += "\n"
#     printed_text += "\end{longtable}"
#     printed_text += "\n\n"
    
#     printed_text += "\clearpage"
#     printed_text += "\n\n"
#     # break
    
# print(printed_text)

# Regression w/ Effect Coding

In [302]:
def closeness_to_vec_effect(label):
    if label == "1. Close":
        return {"x1": 1, "x2": 0}
    elif label == "2. Know each other": ## Base category
        return {"x1": -1, "x2": -1}
    elif label == "3. Don't know each other": 
        return {"x1": 0, "x2": 1}
    elif label == "4. Don't like each other":
        return None
    else:
        return None
    
def authority_to_vec_effect(label):
    if label == '0. Very respect':
        return {"x1": 1, "x2": 0}
    elif label == '1. Respect': 
        return {"x1": 0, "x2": 1}
    elif label == '2. Normal':    ## Base category
        return {"x1": -1, "x2": -1}
    elif label == '3. Not respect':
        # return None
        return {"x1": 1, "x2": 0}
    else:
        return None

In [303]:
def print_weights_effect(outputs, coef_labels):
    coefs = {}
    for i, label in coef_labels.items():
        if i is None:
            continue
            
        for _, row in outputs.iterrows():
            coefs[(label, row["feat"])] = (row[f"coef{i}"], row[f"pval{i}"])

    printed_text = ""
    for g in metric_names:
        # print("\multicolumn{4}{l}{\\textit{"+g+"}} \\\\")
        printed_text += "\multicolumn{4}{l}{\\textit{"+g+"}} \\\\"+"\n"
        for m in metric_names[g]:
            s = f"{metric_names[g][m]} "
            for i, l in coef_labels.items():                    
                if i is None:
                    s += f"& * "
                    continue
                
                val, pval = coefs[(l, m)]
                if l=="Grand Mean":
                    s += f"& {val:.2f} "
                elif pval < 0.05:
                    s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                else:
                    s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
        printed_text += "&  & &\\\\" + "\n"
        printed_text += "\hline" + "\n"
        # print("&  & &\\\\")
        # print("\hline")
    return printed_text, coef_labels

In [304]:
feats = to_features(analysis_values, analysis_labels, "closeness", 0, closeness_to_vec_effect, skips=[])
clse1_effect_outputs = print_anova_test(feats, n=2)
# ["Grand Mean", "0. Very respect", "1. Respect", "", "3. Not respect"], "2. Normal"

coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text1 = print_weights_effect(clse1_effect_outputs, coef_labels)

feats = to_features(analysis_values, analysis_labels, "authority", 0, authority_to_vec_effect, skips=["3. Not respect"])
auth1_effect_outputs = print_anova_test(feats, n=2)

coef_labels = {0:"Grand Mean", 1: "Very respect", 2: "Respect", None: "Normal"}
printed_text2 = print_weights_effect(auth1_effect_outputs, coef_labels)

In [305]:
feats = to_features(analysis_values, analysis_labels, "closeness", 1, closeness_to_vec_effect, skips=[])
clse2_effect_outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text3 = print_weights_effect(clse2_effect_outputs, coef_labels)


feats = to_features(analysis_values, analysis_labels, "authority", 1, authority_to_vec_effect, skips=["0. Very respect"])
auth2_effect_outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", None: "Normal", 2: "Respect", 1: "Not respect"}
printed_text4 = print_weights_effect(auth2_effect_outputs, coef_labels)

error particles_?? wrong shape for coefs
error particles_?? r_matrix performs f_test for using dimensions that are asymptotically non-normal


/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [306]:
feats = to_features(analysis_values, analysis_labels, "closeness", 2, closeness_to_vec_effect, skips=[])
clse3_effect_outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", 1: "Close", None: "Know each other", 2: "Don't know each other"}
printed_text5 = print_weights_effect(clse3_effect_outputs, coef_labels)

feats = to_features(analysis_values, analysis_labels, "authority", 2, authority_to_vec_effect, skips=["0. Very respect"])
auth3_effect_outputs = print_anova_test(feats, n=2)
coef_labels = {0:"Grand Mean", None: "Normal", 2: "Respect", 1: "Not respect"}
printed_text6 = print_weights_effect(auth3_effect_outputs, coef_labels)

/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/Users/imtk/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [307]:
# feats[feats["metric"]=="nsent"].groupby(["x1", "x2"]).count()

In [308]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

table_contents = [
    (printed_text1, printed_text2),
    (printed_text3, printed_text4),
    (printed_text5, printed_text6),
]

# columns = 
# (["Grand Mean", "1. Close", "2. Know each other", "3. Don't know each other"], "2. Know each other")

# ["Grand Mean", "0. Very respect", "1. Respect", "2. Normal", "3. Not respect"], "2. Normal"

printed_text = ""
for section, (v1, v2) in zip(sections, table_contents):
    
    printed_text += "\subsection{"+section+"}"+"\n\n"
    
    printed_text += '''\subsubsection{Closeness}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline
    '''
    
    t1, col1 = v1
    printed_text += "Lexical Features & " + " & ".join(col1.values()) + "\\\\"
        
    printed_text +='''
    \hline
    \endfirsthead

    \endhead
    '''

    
    printed_text += ("\n"+t1).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"

    
    printed_text += '''\clearpage
\subsubsection{Respect}
\\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\\tabcolsep}
}
    \hline
    '''

    t2, col2 = v2
    
    printed_text += "Lexical Features & " + " & ".join(col2.values()) + "\\\\"
        
    printed_text +='''
    \hline
    \endfirsthead

    \endhead
    '''
    
    printed_text += ("\n"+t2).replace("\n", "\n    ")
    printed_text += "\n"
    printed_text += "\end{longtable}"
    printed_text += "\n\n"
    
    printed_text += "\clearpage"
    printed_text += "\n\n"
    # break
    
# print(printed_text)

### Print Results

In [309]:
sections = [
    "Setting 1: Private Conversations with Self-Reported Labels",
    "Setting 2: Public Conversations with Labels from 3rd Party ",
    "Setting 3: Private Conversations with Labels from 3rd Party ",
]

In [310]:
def outputs_to_dict(outputs, coef_labels):
    coefs = {}
    for label in coef_labels.keys():
        for _, row in outputs.iterrows():
            coef_col, pval_col = coef_labels[label]
            if coef_col is None:
                coefs[(label, row["feat"])] = (None, None)
            elif coef_col not in row:
                continue
            else:
                coefs[(label, row["feat"])] = (row[coef_col], row[pval_col])

    return coefs

In [311]:

def get_printed_text_by_section(title, coef_labels, outputs, section_label):
    printed_text = ""
    printed_text += "\subsubsection{"+title+"}"+"\n"
    printed_text += "\\begin{longtable}[h]{"+"\n"
    printed_text += "    p{\dimexpr 0.40\linewidth-2\\tabcolsep}|"+"\n"
    
    ncol = len(coef_labels)
    if ncol==5:
        printed_text += "    p{\dimexpr 0.12\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.12\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.12\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.12\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.12\linewidth-2\\tabcolsep}"+"\n"
    elif ncol==4:
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
    elif ncol==3:
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
        printed_text += "    p{\dimexpr 0.15\linewidth-2\\tabcolsep}"+"\n"
    else:
        assert(False)
        
    printed_text += "}"+"\n"
    printed_text += "    \hline"+"\n"
    printed_text += "    Lexical Features & "+" & ".join(coef_labels)+"\\\\"+"\n"
    printed_text += "    \hline"+"\n"
#     printed_text += "    \endfirsthead"+"\n"
#     printed_text += ""+"\n"
    printed_text += "    \endhead"+"\n"
    printed_text += ""+"\n"
    
    for sec, results in zip(sections, outputs):
        printed_text += "    \multicolumn{4}{l}{\\textit{"+sec+"}} \\\\"+"\n"
        printed_text += "    \hline"+"\n"
        
        for m in metric_names[g]:
            s = f"        {metric_names[g][m]} "
            for l in coef_labels:   
                k = (l, m)
                
                if k not in results:
                    s += f"& - "
                else:
                    val, pval = results[k]
                    if l=="Grand Mean":
                        s += f"& {val:.2f} "
                    elif val is None:
                        s += f"& * " ## Base Category
                    elif pval < 0.05:
                        s += "& \cellcolor{gray!25} "+f"{val:.2f} "
                    else:
                        s += f"& {val:.2f} "
            s += "\\\\"
            # print(s)
            printed_text += s+"\n"
            
        
#         printed_text += "        &  & &\\\\"+"\n"
        printed_text += "    \hline"+"\n"
        printed_text += ""+"\n"
    printed_text += "\label{"+section_label+"}"+"\n"
    printed_text += "\end{longtable}"+"\n"
    
    
    return printed_text

In [313]:
metric_names_code = {
    "Linguistic Complexity": "ling_comp",
    "Pronoun": "pronoun",
    "Sentence-ending Particles": "particle",
    "Sentiment-related": "sentiment",
    "Spelling Variation": "spelling",
}

printed_text = "" 
for g in metric_names:
    if g in ["Conversational Statistics"]:
        continue
    printed_text += "\subsection{"+g+"}"+"\n"+"\n"
    
    ###### Closeness Effect Coding #######
    coef_labels = {
        "Grand Mean": ("coef0", "pval0"),
        "Close": ("coef1", "pval1"),
        "Know each other": (None, None),
        "Don't know each other": ("coef2", "pval2"),
    }
    
    outputs = [
        outputs_to_dict(clse1_effect_outputs, coef_labels),
        outputs_to_dict(clse2_effect_outputs, coef_labels),
        outputs_to_dict(clse3_effect_outputs, coef_labels),
    ]
    
    
    

    printed_text += get_printed_text_by_section(
        "Regression Analysis on Closeness with Effect Coding",
        coef_labels.keys(), 
        outputs,
        "tab:closeness_effect_"+metric_names_code[g]
    )
    
    ###### Closeness Polynomial Coding #######
    
    coef_labels = {
        "Grand Mean": ("coef0", "pval0"),
        "Linear": ("coef1", "pval1"),
        "Quadratic": ("coef2", "pval2"),
    }
    
    outputs = [
        outputs_to_dict(clse1_poly_outputs, coef_labels),
        outputs_to_dict(clse2_poly_outputs, coef_labels),
        outputs_to_dict(clse3_poly_outputs, coef_labels),
    ]
    
    printed_text += get_printed_text_by_section(
        "Regression Analysis on Closeness with Orthogonal Polynomial Coding",
        coef_labels.keys(), 
        outputs,
        "tab:closeness_polynomial_"+metric_names_code[g]
    )
    
    printed_text += "\clearpage"+"\n"
    
    
    ###### Respect Effect Coding #######
    
    coef_labels1 = {
        "Grand Mean": ("coef0", "pval0"),
        "Very respect": ("coef1", "pval1"),
        "Respect": ("coef2", "pval2"),
        "Normal": (None, None),
        "Not respect": ("-", "-"),
    }
    
    coef_labels2 = {
        "Grand Mean": ("coef0", "pval0"),
        "Very respect": ("-", "-"),
        "Respect": ("coef1", "pval1"),
        "Normal": (None, None),
        "Not respect": ("coef2", "pval2"),
    }
    
    outputs = [
        outputs_to_dict(auth1_effect_outputs, coef_labels1),
        outputs_to_dict(auth2_effect_outputs, coef_labels2),
        outputs_to_dict(auth3_effect_outputs, coef_labels2),
    ]
    
    
    
    printed_text += get_printed_text_by_section(
        "Regression Analysis on Respect with Effect Coding",
        coef_labels1.keys(), 
        outputs,
        "tab:respect_effect_"+metric_names_code[g]
    )
    
    ###### Respect Polynomial Coding #######
    
    coef_labels = {
        "Grand Mean": ("coef0", "pval0"),
        "Linear": ("coef1", "pval1"),
        "Quadratic": ("coef2", "pval2"),
    }
    
    outputs = [
        outputs_to_dict(auth1_poly_outputs, coef_labels),
        outputs_to_dict(auth2_poly_outputs, coef_labels),
        outputs_to_dict(auth3_poly_outputs, coef_labels),
    ]
    
    printed_text += get_printed_text_by_section(
        "Regression Analysis on Respect with Orthogonal Polynomial Coding",
        coef_labels.keys(), 
        outputs,
        "tab:respect_polynomial_"+metric_names_code[g]
    )
    
    printed_text += "\clearpage"+"\n"
    
#     break
    

In [314]:
print(printed_text)

\subsection{Linguistic Complexity}

\subsubsection{Regression Analysis on Closeness with Effect Coding}
\begin{longtable}[h]{
    p{\dimexpr 0.40\linewidth-2\tabcolsep}|
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
    p{\dimexpr 0.15\linewidth-2\tabcolsep}
}
    \hline
    Lexical Features & Grand Mean & Close & Know each other & Don't know each other\\
    \hline
    \endhead

    \multicolumn{4}{l}{\textit{Setting 1: Private Conversations with Self-Reported Labels}} \\
    \hline
        Vocabulary size & 70.64 & \cellcolor{gray!25} 1.08 & * & -0.65 \\
        Thai words & 94.68 & 0.25 & * & \cellcolor{gray!25} -0.74 \\
        Non-Thai words & 0.52 & 0.02 & * & 0.02 \\
        Long words & 3.83 & \cellcolor{gray!25} -0.29 & * & -0.07 \\
        Dictionary words & 78.82 & \cellcolor{gray!25} -1.21 & * & \cellcolor{gray!25} 1.21 \\
        Transliteration & 1.39 & 0.13 & * & 0.00 \\
    \hline

    \m